In [6]:
import pandas as pd
import numpy as np
import datetime
from pandas_datareader import data as wb
import statsmodels.api as sm
import numpy_financial as npf

In [7]:
# ++++++++++++++++++++++++++++++++++  ОЦЕНИВАЕМ ИСТИННУЮ СТОИМОСТЬ  +++++++++++++++++++++++++++++++++++++++


def gordon_wacc(work_table, LTGrowth, LTdis, PRICE):

    print('Расчет показателей Gordon, WACC, spl_num, PRICE_model')

    Dtoday = work_table['dividendsPaid'][0] / work_table['numberOfShares'][0]

    # ROE and PAyout Ratio


    payout_ratio = work_table['dividendPayoutRatio'].median()
    sustgrwothrate = work_table['ROE_DuPont'][0] * (1 - payout_ratio)

    ##Cost of equity:
    # Risk Free Rate
    start = datetime.datetime(2019, 2, 1)
    end = datetime.datetime.today().strftime('%Y-%m-%d')

    Treasury = wb.DataReader(['DGS10'], 'fred', start, end)
    RF = float(Treasury.iloc[-1])
    RF = RF / 100
    # print(f'RF {RF}')


    # Market Return
    # Считаем годовую доходность
    data = wb.DataReader('^GSPC', data_source='yahoo', start='2001-1-1')  # нужно определиться с переменной
    data['simple_return'] = (data['Adj Close'] / data['Adj Close'].shift(1)) - 1
    data['simple_return'].replace(np.nan, 0)
    avg_returns_a = data['simple_return'].mean() * 250

    # print(f'Годовая доходность {avg_returns_a}')

    # Drop all Not a number values using drop method.
    ke = RF + (float(work_table['beta'][0]) * (avg_returns_a - RF))

    Gordon = (Dtoday * (1 + sustgrwothrate)) / (ke - sustgrwothrate)


    # ------------------------------------------------------------------------------------------------------------

    work_table['year_index'] = range(1, len(work_table) + 1)[::-1]
    work_table['intercept'] = 1


    # modeling
    freeCashFlow_model = sm.OLS(work_table['freeCashFlow'], work_table[['year_index', 'intercept']]).fit()

    freeCashFlow_pred_lr = freeCashFlow_model.predict([19, 1])[0]  # Прогнозируем через 10 лет

    freeCashFlow_pred_lr5 = freeCashFlow_model.predict([14, 1])[0]  # Прогнозируем через 5 лет
    freeCashFlow_pred_lr4 = freeCashFlow_model.predict([13, 1])[0]
    freeCashFlow_pred_lr3 = freeCashFlow_model.predict([12, 1])[0]
    freeCashFlow_pred_lr2 = freeCashFlow_model.predict([11, 1])[0]
    freeCashFlow_pred_lr1 = freeCashFlow_model.predict([10, 1])[0]

    # CAGR FCF
    CF_forecast = {}

    CF_forecast['next_year1'] = {}
    CF_forecast['next_year1']['freeCashFlow'] = freeCashFlow_pred_lr1

    CF_forecast['next_year2'] = {}
    CF_forecast['next_year2']['freeCashFlow'] = freeCashFlow_pred_lr2

    CF_forecast['next_year3'] = {}
    CF_forecast['next_year3']['freeCashFlow'] = freeCashFlow_pred_lr3

    CF_forecast['next_year4'] = {}
    CF_forecast['next_year4']['freeCashFlow'] = freeCashFlow_pred_lr4

    CF_forecast['next_year5'] = {}
    CF_forecast['next_year5']['freeCashFlow'] = freeCashFlow_pred_lr5

    # print(CF_forecast)

    # print(f'freeCashFlow_pred_lr1 = {freeCashFlow_pred_lr1}')
    # print(f'freeCashFlow_pred_lr2 = {freeCashFlow_pred_lr2}')
    # print(f'freeCashFlow_pred_lr3 = {freeCashFlow_pred_lr3}')
    # print(f'freeCashFlow_pred_lr4 = {freeCashFlow_pred_lr4}')
    # print(f'freeCashFlow_pred_lr5 = {freeCashFlow_pred_lr5}')
    #
    # print(freeCashFlow_pred_lr)


    # ________________________________________  Расчет WACC  _______________________________________________

    WACC = (work_table['mktCap'][0] / (work_table['mktCap'][0] + work_table['netDebt'][0:2].mean())) * ke + (
            work_table['netDebt'][0:2].mean() / (work_table['mktCap'][0] + work_table['netDebt'][0:2].mean())) * (
                   work_table['interestExpense'][0] / work_table['netDebt'][0:2].mean()) \
           * (1 - (work_table['incomeTaxExpense'][0] / work_table['incomeBeforeTax'][0]))

    # print(f'WACC {WACC}')


    # print('wacc of = ' + company + ' is \n' + str((wacc_company * 100)) + '%')

    CF_forec = pd.DataFrame.from_dict(CF_forecast, orient='columns')

    FCF_List = CF_forec.iloc[0:].values.tolist()

    npv = npf.npv(WACC, FCF_List[0])
    # print(f'npv = {npv}')

    Terminal_value_Discounted = freeCashFlow_pred_lr * (1 + LTGrowth) / (LTdis - LTGrowth)

    # print(f'Terminal_value_Discounted = {Terminal_value_Discounted}')

    sum_num = npv + Terminal_value_Discounted + work_table['cash_TTM'][0] - work_table['totalDebt_TTM'][0]  # добавлен кеш и вычесть долг за последний месяц
    # print(f'sum_num = {sum_num}')

    spl_num = sum_num / work_table['numberOfShares'][0]
    # print(f'Estimated price = {spl_num}')

    difference_model_and_market_value = spl_num - PRICE
    # print(f'stock price = {PRICE}')

    FCF_List_plot = pd.DataFrame(FCF_List[0])
    
    # print(f'difference_model_and_market_value = {difference_model_and_market_value}')

    return Gordon, float(WACC), round(spl_num, 2), PRICE, FCF_List_plot, ke
